# Video Games Sales with Ratings

# 1. **Introduction**
🎮 **It Predicts Video Game Sales Based on their Features**

This project aims to predict global video game sales using different machine learning regression models. The dataset includes information about games such as platform, genre, publisher, ratings, critic scores, and user scores. By training models on these features, the goal is to understand which factors influence sales and how accurately we can predict a game's performance. This project also compares multiple models to find the best one and includes clear visualisations to understand the results.

# 2. **Import Libraries**

In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# 3. **Loading the Dataset**

In [46]:
import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv('/kaggle/input/video-games-sales-with-rating-prediction-datast/Video_Games_Sales_as_at_22_Dec_2016.csv')
df.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN


# 4. **Data Exploration & Cleaning**

In [47]:
df = df.dropna(subset=['Global_Sales'])
df = df[df['Year_of_Release'] >= 1985]      
df.reset_index(drop=True, inplace=True)

# 5. **Select Features**

In [48]:
features = ['Platform', 'Genre', 'Publisher', 'Year_of_Release', 
            'Critic_Score', 'User_Score', 'Rating']

df['User_Score'] = pd.to_numeric(df['User_Score'], errors='coerce')
df['Critic_Score'] = pd.to_numeric(df['Critic_Score'], errors='coerce')

data = df[features + ['Global_Sales']]
data = data.dropna()

# 6. **Preprocessing**

In [49]:
X = data[features]
y = data['Global_Sales']

num_features = ['Year_of_Release', 'Critic_Score', 'User_Score']
cat_features = ['Platform', 'Genre', 'Publisher', 'Rating']

preprocessor = ColumnTransformer([
    ('num', StandardScaler(), num_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features)
])

# 7. **Train - Test Split**

In [50]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 8. **Train 3 Models**

## Linear Regression

In [51]:
lr_model = Pipeline([
    ('preprocess', preprocessor),
    ('model', LinearRegression())
])

lr_model.fit(X_train, y_train)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Year_of_Release',
                                                   'Critic_Score',
                                                   'User_Score']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Platform', 'Genre',
                                                   'Publisher', 'Rating'])])),
                ('model', LinearRegression())])

## Decision Tree

In [52]:
dt_model = Pipeline([
    ('preprocess', preprocessor),
    ('model', DecisionTreeRegressor(max_depth=8, random_state=42))
])

dt_model.fit(X_train, y_train)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Year_of_Release',
                                                   'Critic_Score',
                                                   'User_Score']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Platform', 'Genre',
                                                   'Publisher', 'Rating'])])),
                ('model', DecisionTreeRegressor(max_depth=8, random_state=42))])

## Random Forest

In [53]:
rf_model = Pipeline([
    ('preprocess', preprocessor),
    ('model', RandomForestRegressor(n_estimators=200, random_state=42))
])

rf_model.fit(X_train, y_train)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Year_of_Release',
                                                   'Critic_Score',
                                                   'User_Score']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Platform', 'Genre',
                                                   'Publisher', 'Rating'])])),
                ('model',
                 RandomForestRegressor(n_estimators=200, random_state=42))])

# 9. Evaluation Function

In [54]:
def evaluate(model, X_train, X_test, y_train, y_test):
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    results = {
        'MAE Train': mean_absolute_error(y_train, y_pred_train),
        'MAE Test': mean_absolute_error(y_test, y_pred_test),
        'RMSE Train': np.sqrt(mean_squared_error(y_train, y_pred_train)),
        'RMSE Test': np.sqrt(mean_squared_error(y_test, y_pred_test)),
        'R² Train': r2_score(y_train, y_pred_train),
        'R² Test': r2_score(y_test, y_pred_test)
    }

    return pd.DataFrame(results, index=[0])

# 10. Display Model Performance

In [55]:
print("Linear Regression:")
display(evaluate(lr_model, X_train, X_test, y_train, y_test))

print("Decision Tree:")
display(evaluate(dt_model, X_train, X_test, y_train, y_test))

print("Random Forest:")
display(evaluate(rf_model, X_train, X_test, y_train, y_test))

Linear Regression:


,MAE Train,MAE Test,RMSE Train,RMSE Test,R² Train,R² Test
0,0.73192,0.778283,1.847221,1.490215,0.175454,0.182849


Decision Tree:


,MAE Train,MAE Test,RMSE Train,RMSE Test,R² Train,R² Test
0,0.467671,0.706448,0.840168,2.78624,0.829427,-1.856548


Random Forest:


,MAE Train,MAE Test,RMSE Train,RMSE Test,R² Train,R² Test
0,0.211212,0.594678,0.626046,1.844467,0.905291,-0.251833


# 11. Visualisations

## Actual vs Predicted

In [56]:
models = [('Linear Regression', lr_model),
          ('Decision Tree', dt_model),
          ('Random Forest', rf_model)]

plt.figure(figsize=(15, 18))

for i, (name, model) in enumerate(models):
    y_pred = model.predict(X_test)
    plt.subplot(3, 1, i+1)
    sns.scatterplot(x=y_test, y=y_pred)
    plt.xlabel("Actual Sales")
    plt.ylabel("Predicted Sales")
    plt.title(f"{name}: Actual vs Predicted")

## Residual Error Plots

In [57]:
plt.figure(figsize=(15, 18))

for i, (name, model) in enumerate(models):
    y_pred = model.predict(X_test)
    residuals = y_test - y_pred
    plt.subplot(3, 1, i+1)
    sns.histplot(residuals, kde=True)
    plt.title(f"{name} Residual Error Distribution")

## Feature Importance (Random Forest)

In [58]:
rf = rf_model.named_steps['model']
ohe = rf_model.named_steps['preprocess'].named_transformers_['cat']
encoded_cat = ohe.get_feature_names_out(cat_features)

all_features = np.concatenate([num_features, encoded_cat])

importances = rf.feature_importances_
sorted_idx = np.argsort(importances)[-20:]

plt.figure(figsize=(10, 8))
plt.barh(all_features[sorted_idx], importances[sorted_idx])
plt.title("Top 20 Feature Importances (Random Forest)")
plt.show()

# 12. Gradio Interface

In [59]:
import gradio as gr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import warnings
import io
from PIL import Image

# Suppress warnings
warnings.filterwarnings("ignore")

# ==========================================
# 1. SYSTEM SETUP & TRAINING
# ==========================================

class GameSalesPredictor:
    def __init__(self):
        self.models = {}
        self.data = None
        self.unique_vals = {}
        self.is_trained = False
        
    def load_and_train(self):
        print("🔄 System Initializing: Loading Data & Training Models...")
        
        # --- Path Setup ---
        # Checks for Kaggle path first, falls back to dummy if not found
        paths = [
            '/kaggle/input/video-games-sales-with-rating-prediction-datast/Video_Games_Sales_as_at_22_Dec_2016.csv',
            'Video_Games_Sales_as_at_22_Dec_2016.csv'
        ]
        
        file_found = False
        for p in paths:
            try:
                df = pd.read_csv(p)
                file_found = True
                break
            except:
                continue
        
        if not file_found:
            print("⚠️ Data not found. Generating dummy data for UI demonstration.")
            df = self.generate_dummy_data()

        # --- Cleaning & Feature Engineering ---
        df = df.dropna(subset=['Global_Sales'])
        df = df[df['Year_of_Release'] >= 1980]
        # Convert scores
        df['User_Score'] = pd.to_numeric(df['User_Score'], errors='coerce')
        df['Critic_Score'] = pd.to_numeric(df['Critic_Score'], errors='coerce')
        
        # Drop rows with missing critical features for training
        features = ['Platform', 'Genre', 'Publisher', 'Year_of_Release', 'Critic_Score', 'User_Score', 'Rating']
        self.data = df[features + ['Global_Sales']].dropna()
        
        # --- Store Unique Values for UI ---
        # Get top 50 publishers to keep dropdown clean
        top_publishers = self.data['Publisher'].value_counts().nlargest(50).index.tolist()
        
        self.unique_vals = {
            'Platform': sorted(self.data['Platform'].unique().tolist()),
            'Genre': sorted(self.data['Genre'].unique().tolist()),
            'Publisher': sorted(top_publishers),
            'Rating': sorted(self.data['Rating'].unique().tolist())
        }

        # --- Training Pipeline ---
        X = self.data[features]
        y = self.data['Global_Sales']
        
        num_features = ['Year_of_Release', 'Critic_Score', 'User_Score']
        cat_features = ['Platform', 'Genre', 'Publisher', 'Rating']

        preprocessor = ColumnTransformer([
            ('num', StandardScaler(), num_features),
            ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features)
        ])

        # Define Models
        models_to_train = {
            "🌲 Random Forest (Best)": RandomForestRegressor(n_estimators=50, random_state=42),
            "📈 Linear Regression": LinearRegression(),
            "🌿 Decision Tree": DecisionTreeRegressor(max_depth=10, random_state=42)
        }

        for name, model_inst in models_to_train.items():
            pipeline = Pipeline([
                ('preprocess', preprocessor),
                ('model', model_inst)
            ])
            pipeline.fit(X, y)
            self.models[name] = pipeline
            print(f"✅ {name} trained.")

        self.is_trained = True
        print("🚀 System Ready!")

    def generate_dummy_data(self):
        # Fallback generator
        return pd.DataFrame({
            'Name': [f'Game {i}' for i in range(100)],
            'Platform': ['PS4', 'XOne', 'PC', 'WiiU', '3DS'] * 20,
            'Year_of_Release': np.random.randint(2000, 2020, 100),
            'Genre': ['Action', 'Sports', 'Shooter', 'Role-Playing', 'Misc'] * 20,
            'Publisher': ['Nintendo', 'EA', 'Activision', 'Ubisoft', 'Sony'] * 20,
            'Global_Sales': np.random.rand(100) * 5,
            'Critic_Score': np.random.randint(50, 99, 100),
            'User_Score': np.random.rand(100) * 10,
            'Rating': ['E', 'T', 'M', 'E10+'] * 25
        })

    def predict(self, model_name, platform, genre, publisher, year, critic, user, rating):
        if not self.is_trained:
            return "System Loading...", None

        # Prepare Input
        input_df = pd.DataFrame({
            'Platform': [platform],
            'Genre': [genre],
            'Publisher': [publisher],
            'Year_of_Release': [year],
            'Critic_Score': [critic],
            'User_Score': [user],
            'Rating': [rating]
        })

        # Prediction
        model = self.models[model_name]
        pred_value = model.predict(input_df)[0]
        pred_value = max(0, pred_value) # No negative sales

        # --- Generate Context Plot ---
        # Compare prediction vs Average sales of that Genre
        genre_avg = self.data[self.data['Genre'] == genre]['Global_Sales'].mean()
        all_avg = self.data['Global_Sales'].mean()
        
        fig, ax = plt.subplots(figsize=(6, 4))
        categories = ['Your Game', f'{genre} Avg', 'Global Avg']
        values = [pred_value, genre_avg, all_avg]
        colors = ['#6b46c1', '#a0aec0', '#cbd5e0'] # Purple, Gray, Light Gray
        
        bars = ax.bar(categories, values, color=colors)
        ax.set_ylabel('Sales (Millions)')
        ax.set_title(f'Performance Context ({genre} Genre)')
        
        # Add values on top of bars
        for bar in bars:
            height = bar.get_height()
            ax.annotate(f'{height:.2f}M', xy=(bar.get_x() + bar.get_width() / 2, height),
                        xytext=(0, 3), textcoords="offset points",
                        ha='center', va='bottom')
        
        plt.tight_layout()
        
        return f"{pred_value:.2f} Million Units", fig

    def get_data_preview(self):
        return self.data.head(10)

    def get_correlation_plot(self):
        if self.data is None: return None
        fig, ax = plt.subplots(figsize=(8, 6))
        numeric_df = self.data.select_dtypes(include=[np.number])
        sns.heatmap(numeric_df.corr(), annot=True, cmap='coolwarm', ax=ax)
        return fig

# Initialize Logic
system = GameSalesPredictor()
system.load_and_train()

# ==========================================
# 2. PROFESSIONAL GRADIO UI
# ==========================================

# Modern Custom CSS
custom_css = """
.gradio-container {background-color: #F7FAFC}
h1 {text-align: center; color: #2D3748; font-family: 'Arial', sans-serif;}
.stat-box {background: white; padding: 20px; border-radius: 10px; box-shadow: 0 4px 6px rgba(0,0,0,0.1); text-align: center;}
.stat-value {font-size: 2em; font-weight: bold; color: #5A67D8;}
.stat-label {color: #718096; font-size: 0.9em; text-transform: uppercase; letter-spacing: 1px;}
"""

with gr.Blocks(theme=gr.themes.Soft(), css=custom_css, title="Game Sales AI") as demo:
    
    gr.Markdown("# 🎮 Video Game Market Intelligence")
    gr.Markdown("### AI-Powered Sales Prediction & Analytics Dashboard")
    
    with gr.Tabs():
        # === TAB 1: PREDICTION ENGINE ===
        with gr.TabItem("🚀 Prediction Engine"):
            with gr.Row():
                # LEFT COLUMN: INPUTS
                with gr.Column(scale=1):
                    gr.Markdown("### ⚙️ Game Configuration")
                    
                    with gr.Group():
                        sel_model = gr.Radio(
                            choices=list(system.models.keys()), 
                            value="🌲 Random Forest (Best)", 
                            label="Select AI Model"
                        )
                    
                    with gr.Accordion("📝 Game Details", open=True):
                        in_platform = gr.Dropdown(system.unique_vals['Platform'], label="Platform", value="PS4")
                        in_genre = gr.Dropdown(system.unique_vals['Genre'], label="Genre", value="Action")
                        in_publisher = gr.Dropdown(system.unique_vals['Publisher'], label="Publisher", value="Nintendo")
                        in_year = gr.Slider(1980, 2025, step=1, value=2016, label="Release Year")
                    
                    with gr.Accordion("⭐ Reception & Ratings", open=True):
                        in_rating = gr.Dropdown(system.unique_vals['Rating'], label="ESRB Rating", value="E")
                        in_critic = gr.Slider(0, 100, step=1, value=80, label="Critic Score (Metacritic)")
                        in_user = gr.Slider(0, 10, step=0.1, value=7.5, label="User Score")
                        
                    btn_run = gr.Button("🔮 Generate Prediction", variant="primary", size="lg")

                # RIGHT COLUMN: RESULTS
                with gr.Column(scale=1):
                    gr.Markdown("### 📊 Market Forecast")
                    
                    # Highlighted Result Box
                    with gr.Group():
                        out_text = gr.Textbox(label="Predicted Global Sales", lines=1, text_align="center")
                    
                    # Visual Chart
                    out_plot = gr.Plot(label="Market Context Analysis")
                    
                    gr.Markdown("""
                    > **Note:** Predictions are based on historical data up to 2016. 
                    > The 'Context Analysis' compares your game against historical averages for the selected genre.
                    """)

        # === TAB 2: DATA INTELLIGENCE ===
        with gr.TabItem("📈 Data Intelligence"):
            gr.Markdown("### Training Data Overview")
            with gr.Row():
                with gr.Column():
                    gr.Markdown("**Sample Data**")
                    gr.Dataframe(value=system.get_data_preview, interactive=False)
                with gr.Column():
                    gr.Markdown("**Feature Correlation Heatmap**")
                    gr.Plot(value=system.get_correlation_plot)

    # === WIRING ===
    btn_run.click(
        fn=system.predict,
        inputs=[sel_model, in_platform, in_genre, in_publisher, in_year, in_critic, in_user, in_rating],
        outputs=[out_text, out_plot]
    )

# Run the app
if __name__ == "__main__":
    demo.launch()

🔄 System Initializing: Loading Data & Training Models...
✅ 🌲 Random Forest trained.
✅ 📈 Linear Regression trained.
✅ 🌿 Decision Tree trained.
* Running on local URL:  http://127.0.0.1:7862
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://8bb3f9ab552b437dad.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
